In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
from datasets import load_dataset
import pandas as pd
import numpy as np
from src.tfr_decoding.finepref_sample import sample # new sampling method
from src.utils.samp_utils import inpsampall, dset_randsamp   
from src.tfr_decoding.shp_modeling import T5BinaryClassifier
from prefix_sampling import PrefixSampler, test_baseline, test_pfsample, test_finesample, test_apsample

2023-05-15 17:33:30.611982: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-15 17:33:30.612006: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
# first load relevant models
device = 'cuda:0' # if you have a GPU
pfmod_path = "./lightning_logs/version_4/checkpoints/epoch=2-step=23950.ckpt"

# get generation model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")

qpref = T5BinaryClassifier.load_from_checkpoint(pfmod_path)
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto")#.to(device)
#model.sample = sample.__get__(model)
model.tokenizer = tokenizer
model.tok = tokenizer
pfname = 'stanfordnlp/SteamSHP-flan-t5-large'
max_len = 512
learning_rate = 3e-5
preftok = T5Tokenizer.from_pretrained(pfname)

model.qualitypref = qpref
#model.stdpref = T5BinaryClassifier.load_from_checkpoint("./lightning_logs/version_11/checkpoints/epoch=3-step=3359.ckpt")
# set up relevant mdoels
model.downmetric = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)
model.downtok = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
model.eval()

In [4]:
# load up our dataset
elidf = pd.read_json("output/elidataset.jsonl", orient="records", lines="true")
# start from 15000 for latest exploration
elidf = elidf.drop_duplicates(subset="history").iloc[15000:15100]

pfsampler = PrefixSampler(model)
inplist = list(elidf['history'])

In [ ]:
with torch.no_grad():
    finesample_df = test_finesample(inplist, pfsampler, 9, 3, 6, 5)
    finesample_df.to_json("output/hparam_explore/fsamp.jsonl", orient="records", lines=True)

In [ ]:
with torch.no_grad():
    finesample_df = test_apsample(inplist[:10], pfsampler, 0.85, 3, 1, 3)


In [ ]:
with torch.no_grad():
    finesample_df = test_pfsample(inplist[:10], pfsampler, 0.85, 3, 1, 3)

In [20]:
finesample_df['budgets'].mean()

137.4

In [3]:
a1 = pd.read_json("output/pfsample/finesamp.jsonl", orient="records", lines=True)


In [5]:
a1.budgets.mean()

115.71

In [ ]:
adaptbase_df = test_baseline(inplist, pfsampler, .85, 4)
pfsample_df = test_pfsample(inplist, pfsampler, 18, [10, 20])

adaptbase_df.to_json("output/pfsample/abase.jsonl", lines=True, orient='records')
pfsample_df.to_json("output/pfsample/pfsample.jsonl", lines=True, orient='records')

monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp


In [8]:
print(adaptbase_df['budgets'].mean())
print(adaptbase_df['scos'].mean())

130.4
0.809410572052002


In [12]:
print(pfsample_df['budgets'].mean())
print(pfsample_df['scos'].mean())

253.6
0.8504377722740173


In [26]:
adaptbase_df = test_baseline(inplist, pfsampler, .85, 2)

monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp
monkeysamp


In [27]:
print(adaptbase_df['budgets'].mean())
print(adaptbase_df['scos'].mean())

129.2
0.8531928896903992
